In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from transformers import (AutoModel,AutoModelForMaskedLM, AutoTokenizer, LineByLineTextDataset,
                         DataCollatorForLanguageModeling,Trainer, TrainingArguments)

In [3]:
train_data = pd.read_csv('../input/data-for-distilation/Clinc_Train.csv')
test_data = pd.read_csv('../input/data-for-distilation/Clinc_valid.csv')


data = pd.concat([train_data,test_data])

text  = '\n'.join(data.Text.tolist())

with open('text.txt','w') as f:
    f.write(text)

In [4]:
model_name = '../input/huggingfacedebertav3variants/deberta-v3-small'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at ../input/huggingfacedebertav3variants/deberta-v3-small were not used when initializing DebertaV2ForMaskedLM: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'deberta.embeddings.position_embeddings.weight']
- This IS expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertFo

In [5]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention train text file here
    block_size=128)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args =TrainingArguments(
    output_dir="./Debertav3_small", #select model path
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=12,
#     save_steps=65,
    save_total_limit=1,
    prediction_loss_only=True,
    report_to = "none")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset)

In [ ]:
trainer.train()

Step,Training Loss
500,3.967000
1000,2.444600
1500,2.121000
2000,1.934600
